## Convert Black Marble HDF5 to Geotiff

Using script [provided](https://blackmarble.gsfc.nasa.gov/tools/OpenHDF5.py) by Black Marble and adapted further here. 

In [49]:
import gdal, os

## List input raster files

In [59]:
imgdir = '/Users/lestes/Dropbox/data/imagery/nightlights/'
imgfiles = [f'{imgdir}{img}' for img in os.listdir(imgdir) if ".h5" in img]
imgfiles[0:10]

['/Users/lestes/Dropbox/data/imagery/nightlights/VNP46A3.A2018091.h10v04.001.2021125101408.h5',
 '/Users/lestes/Dropbox/data/imagery/nightlights/VNP46A3.A2017060.h10v04.001.2021125065343.h5',
 '/Users/lestes/Dropbox/data/imagery/nightlights/VNP46A3.A2021335.h10v04.001.2022008112402.h5',
 '/Users/lestes/Dropbox/data/imagery/nightlights/VNP46A3.A2020306.h10v04.001.2021126183147.h5',
 '/Users/lestes/Dropbox/data/imagery/nightlights/VNP46A3.A2012214.h10v04.001.2021118153414.h5',
 '/Users/lestes/Dropbox/data/imagery/nightlights/VNP46A3.A2012306.h10v04.001.2021118161312.h5',
 '/Users/lestes/Dropbox/data/imagery/nightlights/VNP46A3.A2017091.h10v04.001.2021125100540.h5',
 '/Users/lestes/Dropbox/data/imagery/nightlights/VNP46A3.A2014305.h10v04.001.2021126141136.h5',
 '/Users/lestes/Dropbox/data/imagery/nightlights/VNP46A3.A2019091.h10v04.001.2021125101754.h5',
 '/Users/lestes/Dropbox/data/imagery/nightlights/VNP46A3.A2021121.h10v04.001.2021225123453.h5']

## Translation function

In [62]:
def translate_hdf5(imgfile, outfolder):

        #print(rasterFiles)

        #Get File Name Prefix
        imagepre = os.path.basename(imgfile)[:-3]
        print(imagepre)
        
        file_ext = "_BBOX.tif"

        ## Open HDF file
        hdflayer = gdal.Open(imgfile, gdal.GA_ReadOnly)

        #print (hdflayer.GetSubDatasets())

        # Open raster layer
        #hdflayer.GetSubDatasets()[0][0] - for first layer
        #hdflayer.GetSubDatasets()[1][0] - for second layer ...etc
        subhdflayer = hdflayer.GetSubDatasets()[0][0]
        rlayer = gdal.Open(subhdflayer, gdal.GA_ReadOnly)
        #outputName = rlayer.GetMetadata_Dict()['long_name']

        #Subset the Long Name
        outname = os.path.basename(subhdflayer[92:])
        outname = imagepre + file_ext
        # print(outname)

        outfile = os.path.join(outfolder, outname)

        #collect bounding box coordinates
        horiz_tile = int(rlayer.GetMetadata_Dict()["HorizontalTileNumber"])
        vert_tile = int(rlayer.GetMetadata_Dict()["VerticalTileNumber"])

        westbound = (10*horiz_tile) - 180
        northbound = 90-(10*vert_tile)
        eastbound = westbound + 10
        southbound = northbound - 10

        EPSG = "-a_srs EPSG:4326" #WGS84

        transl_opt_text = (
            EPSG+" -a_ullr " + str(westbound) + " " + \
            str(northbound) + " " + str(eastbound) + " " + str(southbound)
        )

        transl_opts = gdal.TranslateOptions(gdal.ParseCommandLine(transl_opt_text))
        gdal.Translate(outfile, rlayer, options=transl_opts)

## Process in loop

In [ ]:
for file in imgfiles:
    print(file)
    translate_hdf5(file, imgdir)